In [1]:
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
from tqdm.auto import tqdm

In [3]:
import re

In [4]:
data = pd.read_csv("./uiks-utf8.csv")

In [5]:
data.head()

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes
0,Республика Адыгея (Адыгея),1 Адыгейская,УИК №1,2277,1925,1925,0,1749,176
1,Республика Адыгея (Адыгея),1 Адыгейская,УИК №2,2694,2290,2290,7,2117,166
2,Республика Адыгея (Адыгея),1 Адыгейская,УИК №3,2865,2441,2441,11,2171,259
3,Республика Адыгея (Адыгея),1 Адыгейская,УИК №4,2054,1669,1669,8,1381,280
4,Республика Адыгея (Адыгея),1 Адыгейская,УИК №5,714,607,607,1,473,133


In [7]:
locations = pd.read_csv("cik_uik.csv.gz")

In [8]:
locations = locations[locations.type_ik == 'uik'].fillna("")

In [9]:
lat = []
lon = []
miss = 0
hits = 0

def get_lat_lon(x):
    normal_region = re.sub("\(.+?\)", "", x.region_name) ## removing (smth) from region_name to make it compatible with cik_uik addresses
    normal_uik    = x.uik_name.replace("УИК", "Участковая избирательная комиссия") ## exapanding uik abbrev.
    retrieval     = locations[locations.address.str.contains(normal_region)]
    retrieval     = retrieval[retrieval.name.str.contains(normal_uik)]
    if len(retrieval) != 1:
        return (None, None)
    else:
        return (retrieval.iloc[0]['lat_voteroom'], retrieval.iloc[0]['lon_voteroom'])
for x in tqdm(data.itertuples(), total=len(data)):
    latitude, longitude = get_lat_lon(x)
    if latitude is None:
        miss += 1
        lat.append(None)
        lon.append(None)
    else:
        hits += 1
        lat.append(latitude)
        lon.append(longitude)

In [10]:
print(f"{(hits/len(data)) * 100:.4f}% locations found")

80.0661% locations found


In [11]:
data['latitude'] = lat
data['longitude'] = lon

In [12]:
data['longitude'] = data['longitude'].apply(lambda x: float(x) if x else float("NaN"))

In [13]:
data['latitude'] = data['latitude'].apply(lambda x: float(x) if x else float("NaN"))

In [14]:
data.to_csv("uiks-sig.csv", index=False, encoding='utf-8-sig')
data.to_csv("uiks-utf8.csv", index=False, encoding='utf-8')

In [15]:
data.head()

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes,latitude,longitude
0,Республика Адыгея (Адыгея),1 Адыгейская,УИК №1,2277,1925,1925,0,1749,176,NaN,NaN
1,Республика Адыгея (Адыгея),1 Адыгейская,УИК №2,2694,2290,2290,7,2117,166,44.983019,39.103872
2,Республика Адыгея (Адыгея),1 Адыгейская,УИК №3,2865,2441,2441,11,2171,259,NaN,NaN
3,Республика Адыгея (Адыгея),1 Адыгейская,УИК №4,2054,1669,1669,8,1381,280,44.887695,39.202791
4,Республика Адыгея (Адыгея),1 Адыгейская,УИК №5,714,607,607,1,473,133,44.883646,39.183058


In [17]:
data.describe()

,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes,latitude,longitude
count,96765.000000,96765.000000,96765.000000,96765.000000,96765.000000,96765.000000,77120.000000,77120.000000
mean,1116.102010,755.520147,754.653087,6.251744,589.790926,158.610417,53.695701,56.515429
std,806.629202,569.552504,568.642758,15.386206,479.266967,145.516624,4.725654,26.453522
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.951224,-179.484049
25%,356.000000,279.000000,279.000000,0.000000,219.000000,38.000000,51.632924,38.995941
50%,1025.000000,645.000000,644.000000,3.000000,461.000000,121.000000,54.623902,46.668168
75%,1774.000000,1096.000000,1094.000000,8.000000,832.000000,246.000000,56.170335,61.376179
max,8306.000000,8306.000000,8265.000000,1142.000000,7202.000000,2631.000000,74.225670,179.256352
